In [1]:
# Score all words
import sys; sys.path.append('../..')
from abslithist import *

In [2]:
corpora = ['CanonFiction']

In [3]:
def get_all_scores(corpora,field='Abs-Conc.Median'):
    norms = get_allnorms()
    words = set(norms.index)
    words -= get_stopwords()
    norms = norms.loc[words]
    for cname in corpora:
        C=lltk.load(cname)
        mfw=set(C.mfw(n=25000))
        for t in tqdm(C.texts(),desc='Scoring texts'):
            freqs=t.freqs()
            period=to_field_period(t.year)
            tfield=field+'.'+period
            w2score=dict(zip(norms.index, norms[tfield]))
            if freqs:
                for w,c in freqs.items():
                    if not w in mfw: continue
                    if not w in words: continue
                    wscore=w2score[w]
                    for i in range(c):
                        yield {'year':t.year,'score':wscore}
#                         yield {'id':t.id, 'score':wscore}
#                         yield {'corpus':cname, 'id':t.id, 'word':w, 'score':wscore}

In [4]:
next(get_all_scores(corpora))

[CanonFiction] Loading MFW from ~/lltk_data/corpora/canon_fiction/data/mfw/eec2d94bb3c5.ft


Scoring texts:   0%|          | 0/1809 [00:00<?, ?it/s]


{'year': 1958, 'score': -0.2489892948268575}

In [5]:
ofn='../../data/scores/tilt.scores.ft'
if os.path.exists(ofn):
    all_scores=pd.read_feather(ofn)
else:
    all_scores = pd.DataFrame(get_all_scores(corpora))
    all_scores.to_feather(ofn)

In [8]:
all_scores

,year,score
0,1958,-0.248989
1,1958,-0.248989
2,1958,-0.248989
3,1958,-0.248989
4,1958,-0.248989
...,...,...
47868906,-400,0.477792
47868907,-400,-1.219819
47868908,-400,-0.542942
47868909,-400,0.310845


In [9]:
dfq=all_scores.query('1600<=year<2000')
dfq

,year,score
0,1958,-0.248989
1,1958,-0.248989
2,1958,-0.248989
3,1958,-0.248989
4,1958,-0.248989
...,...,...
47840522,1611,1.655414
47840523,1611,0.806341
47840524,1611,NaN
47840525,1611,1.193068


In [10]:
dfq['period']=dfq.year.progress_apply(to_cent)

100%|██████████| 45187068/45187068 [00:29<00:00, 1550551.89it/s]


In [11]:
dfqg=dfq.groupby('period').sample(n=1000000)
dfqg

,year,score,period
47082276,1621,-0.477538,C17
1021887,1696,0.188569,C17
37144618,1600,0.150548,C17
33959708,1635,-0.108174,C17
3079883,1627,-1.072318,C17
...,...,...,...
31736566,1939,-0.401427,C20
44682600,1987,0.474647,C20
44477591,1958,-0.987448,C20
28827236,1940,1.622428,C20


In [ ]:
lltk

In [ ]:
# !ls -ltrh ../../data/scores/